## Tatouage visible (Watermark texte)

Dans cette cellule, on utilise la bibliothèque OpenCV (`cv2`) pour appliquer un tatouage **visible** sur une image, c'est-à-dire un texte qu'on peut clairement voir.

Voici les étapes importantes :

1. **Chargement de l’image :**
   On lit l’image originale avec `cv2.imread()`. Si l’image n’est pas trouvée, un message d’erreur est affiché.

2. **Définition des paramètres du texte :**
   - `text` : le contenu du tatouage (ex. : un nom ou une date)
   - `position` : la position du texte (coordonnées X, Y sur l’image)
   - `font_scale` : la taille du texte
   - `color` : la couleur du texte (ici rouge `(0, 0, 255)`, car OpenCV utilise l’ordre BGR)
   - `thickness` : l’épaisseur du texte

3. **Ajout du texte sur l’image :**
   La fonction `cv2.putText()` écrit le texte directement sur l’image.

4. **Sauvegarde de l’image modifiée :**
   L’image avec le tatouage est enregistrée grâce à `cv2.imwrite()`.

💡 Ce type de tatouage est **facile à mettre en place** et permet de **revendiquer la propriété** d’une image. Mais il peut être effacé ou rogné si on ne le place pas stratégiquement.


In [ ]:
import cv2

def apply_text_watermark(image_path, output_path, text, position, font_scale, color, thickness):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image non trouvée.")

    font = cv2.FONT_HERSHEY_SIMPLEX

    cv2.putText(image, text, position, font, font_scale, color, thickness, cv2.LINE_AA)

    cv2.imwrite(output_path, image)
    print(f"Tatouage texte appliqué et sauvegardé : {output_path}")

apply_text_watermark(
    image_path=r"C:\Users\User\Desktop\tatouage\ocean1.jpg",
    output_path=r"C:\Users\User\Desktop\tatouage\ocean1_tatoue.jpg",
    text=" Dhia & Med Amine 2025",
    position=(200,600),
    font_scale=4,
    color=(0, 0, 255),  
    thickness=5
)


Tatouage texte appliqué et sauvegardé : C:\Users\User\Desktop\tatouage\ocean1_tatoue.jpg


## Tatouage invisible (Stéganographie dans une image)

Dans cette partie, on va **cacher un message secret dans une image**, sans que cela soit visible à l'œil nu.  
Ce procédé s’appelle la **stéganographie** : c’est une technique utilisée pour dissimuler des informations dans un support (ici, une image).

Voici comment cela fonctionne, étape par étape :

---

### 🔧 1. Conversion du message en binaire

La fonction `text_to_binary(text)` transforme chaque caractère du message en sa représentation binaire sur 8 bits (ex. : "A" → `01000001`).  
On obtient une longue chaîne de 0 et de 1.

---

### 📥 2. Encodage dans l’image

La fonction `encode_message()` cache le message binaire dans l’image :

- On lit l’image avec `cv2.imread()`
- On remplace **le bit de poids faible (LSB)** de chaque composante de couleur (R, G, B) d’un pixel par un bit du message.  
  ⚠️ Le bit modifié change peu la couleur, donc c’est **invisible** à l’œil nu.
- On ajoute à la fin un **marqueur spécial** (`1111111111111110`) pour signaler la fin du message.

💡 Si l’image est trop petite pour contenir tout le message, un message d’erreur est affiché.

---

### 📤 3. Décodage du message

La fonction `decode_message()` lit les bits de poids faible de chaque pixel de l’image tatouée, jusqu’à retrouver le marqueur de fin.

Ensuite, elle transforme les bits en caractères pour récupérer le message caché.

---

### 🎯 Pourquoi utiliser un tatouage invisible ?

- Il permet de **protéger une image sans modifier son apparence**.
- On peut y inclure des **droits d’auteur, des signatures numériques, ou des infos secrètes**.
- Il est plus difficile à supprimer qu’un tatouage visible… mais aussi plus complexe à mettre en œuvre.


In [ ]:
import cv2

def text_to_binary(text):
    return ''.join([format(ord(c), '08b') for c in text])

def binary_to_text(binary):
    chars = [binary[i:i+8] for i in range(0, len(binary), 8)]
    return ''.join([chr(int(c, 2)) for c in chars if int(c, 2) != 0])

def encode_message(image_path, message, output_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image non trouvée.")
    
    binary_message = text_to_binary(message) + '1111111111111110'  

    data_index = 0
    binary_len = len(binary_message)

    for row in image:
        for pixel in row:
            for i in range(3):  
                if data_index < binary_len:
                    pixel[i] = (pixel[i] & ~1) | int(binary_message[data_index])
                    data_index += 1

    if data_index < binary_len:
        raise ValueError("Message trop long pour cette image.")

    cv2.imwrite(output_path, image)
    print(f"Message caché et image sauvegardée : {output_path}")

def decode_message(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image non trouvée.")
    
    binary_data = ""
    for row in image:
        for pixel in row:
            for i in range(3):  
                binary_data += str(pixel[i] & 1)

    end_marker = '1111111111111110'
    end_index = binary_data.find(end_marker)
    if end_index != -1:
        binary_data = binary_data[:end_index]
    
    return binary_to_text(binary_data)
    from invisible_watermark import encode_message, decode_message

encode_message(
    image_path=r"C:\Users\User\Desktop\tatouage\ocean1.jpg",
    message="© Dhia & Med Amine 2025 - All rights reserved.",
    output_path=r"C:\Users\User\Desktop\tatouage\ocean1_invisible.png"
)

msg = decode_message(r"C:\Users\User\Desktop\tatouage\ocean1_invisible.png")
print("Message extrait :", msg)



Message caché et image sauvegardée : C:\Users\User\Desktop\tatouage\ocean1_invisible.png
Message extrait : © Dhia & Med Amine 2025 - All rights reserved.


Décodage LSB (Least Significant Bit)
Dans cette cellule, nous utilisons la méthode de codage LSB (Least Significant Bit) pour extraire un message caché dans une image. Le principe est d'utiliser les bits les moins significatifs des pixels d'une image pour y insérer discrètement des données, généralement du texte.

Voici les étapes importantes du code :

Chargement de l’image :
La fonction cv2.imread(image_path) permet de charger l'image spécifiée par le chemin image_path. Si l'image n'existe pas ou ne peut pas être lue, une exception (ValueError) est levée avec le message "Image non trouvée".

Parcours des pixels de l'image :

Le code itère à travers chaque ligne (row) de l'image.

Ensuite, il parcourt chaque pixel de la ligne. Un pixel dans une image RGB est une liste de trois valeurs, chacune représentant la composante de couleur (rouge, vert, bleu).

Extraction des bits LSB :

Pour chaque pixel, la fonction examine les trois composantes (rouge, vert, bleu) et applique un masque (& 1) sur chaque composante pour obtenir le dernier bit (le LSB). Le résultat de cette opération est soit 0 soit 1, qui est ajouté à la chaîne de données binaires binary_data.

Recherche du marqueur de fin :

Le code cherche un marqueur spécial ('1111111111111110') dans les données binaires extraites. Ce marqueur indique la fin du message caché.

Si ce marqueur est trouvé, l'extraction des données binaires s'arrête à cet endroit, et la chaîne binaire est tronquée pour ne conserver que les bits avant ce marqueur.

Conversion des données binaires en texte :
La fonction binary_to_text(binary_data) est utilisée pour convertir les données binaires extraites en texte lisible. Cela permet de récupérer le message caché dans l'image.

💡 Ce type de méthode de codage LSB est largement utilisé pour cacher des informations dans des images de manière discrète. L'avantage est que les modifications sont invisibles à l'œil nu, mais l'inconvénient est que l'image peut être altérée si elle est compressée ou modifiée.

In [ ]:
def decode_lsb(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image non trouvée.")
    binary_data = ""
    for row in image:
        for pixel in row:
            for i in range(3):
                binary_data += str(pixel[i] & 1)
    end_marker = '1111111111111110'
    end_index = binary_data.find(end_marker)
    if end_index != -1:
        binary_data = binary_data[:end_index]
    return binary_to_text(binary_data)


Embedding de Watermark avec DCT (Discrete Cosine Transform)
Dans cette cellule, on applique une méthode de tatouage invisible à une image en utilisant la Transformation en Cosinus Discret (DCT). Cette méthode permet d'incorporer un message secret dans l'image en modifiant les coefficients DCT de certains blocs de pixels.

Voici les étapes importantes du code :

Fonctions DCT et IDCT :

dct2(block) : Cette fonction applique la Transformation en Cosinus Discret (DCT) sur un bloc de pixels 8x8. Cela permet de passer de l'espace des pixels à l'espace des fréquences, où les informations sont plus facilement manipulables.

idct2(block) : Cette fonction applique la transformation inverse (IDCT) pour revenir à l'espace des pixels après avoir modifié les coefficients DCT.

Chargement de l’image :

L'image est chargée en niveau de gris avec cv2.imread(image_path, cv2.IMREAD_GRAYSCALE). Si l'image n'est pas trouvée, une exception (ValueError) est levée avec le message "Image non trouvée".

Conversion du message en binaire :

Le message à intégrer dans l'image est converti en binaire avec text_to_binary(message). Ensuite, un marqueur de fin '1111111111111110' est ajouté pour indiquer où se termine le message caché.

Préparation de l'image pour le tatouage :

On récupère les dimensions de l'image avec img.shape (hauteur h et largeur w).

Une copie de l'image est faite pour y ajouter le watermark sans modifier l'originale.

Un indice data_index est initialisé à 0 pour parcourir chaque bit du message binaire.

Insertion du message binaire dans les coefficients DCT :

Le processus parcourt l'image en blocs 8x8 de pixels.

Pour chaque bloc, la DCT est appliquée avec dct2(block), puis un bit du message est inséré dans le coefficient DCT central (en général, la position [4, 4]).

Le bit du message remplace le bit de poids faible (LSB) du coefficient DCT central avec dct_block[4, 4] = (dct_block[4, 4] & ~1) | bit.

Ensuite, la transformation inverse (IDCT) est appliquée avec idct2(dct_block) pour revenir à l'image modifiée.

Ce processus continue jusqu'à ce que tous les bits du message soient insérés ou que l'image soit parcourue.

Enregistrement de l'image modifiée :

L'image avec le watermark est enregistrée à l'emplacement spécifié par output_path avec cv2.imwrite(output_path, watermarked).

Un message de succès est affiché pour informer que le watermark a été appliqué correctement.

💡 Cette méthode de tatouage est très discrète et difficile à détecter, car elle n'affecte que les coefficients de fréquence de l'image, ce qui permet de cacher des informations sans altérer visiblement l'image. Cependant, l'image peut être affectée par une compression ou une transformation importante qui dégrade le watermark.

In [ ]:
def dct2(block):
    return cv2.dct(np.float32(block))

def idct2(block):
    return cv2.idct(np.float32(block))

def embed_dct_watermark(image_path, message, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("Image non trouvée")
    msg_bin = text_to_binary(message) + '1111111111111110'
    h, w = img.shape
    watermarked = np.copy(img)
    data_index = 0
    for i in range(0, h, 8):
        for j in range(0, w, 8):
            if data_index >= len(msg_bin):
                break
            block = img[i:i+8, j:j+8]
            if block.shape != (8, 8):
                continue
            dct_block = dct2(block)
            bit = int(msg_bin[data_index])
            dct_block[4, 4] = (dct_block[4, 4] & ~1) | bit
            watermarked[i:i+8, j:j+8] = idct2(dct_block)
            data_index += 1
    cv2.imwrite(output_path, watermarked)
    print("✅ Watermark DCT appliqué :", output_path)



Extraction de Watermark avec DCT (Discrete Cosine Transform)
Dans cette cellule, on extrait un message caché dans une image à l'aide de la Transformation en Cosinus Discret (DCT), utilisée pour insérer le watermark. Cette méthode permet de récupérer les informations cachées dans les coefficients DCT des blocs d'une image.

Voici les étapes importantes du code :

Chargement de l’image :

L'image est chargée en niveaux de gris avec cv2.imread(image_path, cv2.IMREAD_GRAYSCALE). Si l'image ne peut pas être lue (par exemple si elle n'existe pas), une exception (ValueError) est levée avec le message "Image non trouvée".

Initialisation des variables :

On récupère les dimensions de l'image avec img.shape (hauteur h et largeur w).

Une variable binary_msg est initialisée comme une chaîne vide, elle servira à stocker les bits extraits du watermark.

Parcours de l'image par blocs 8x8 :

Le code parcourt l'image en blocs de 8x8 pixels. À chaque itération, il extrait un bloc de l'image.

Si le bloc n'a pas la taille de 8x8 pixels, il est ignoré.

Extraction des bits du watermark :

Pour chaque bloc de 8x8 pixels, la DCT est appliquée avec dct2(block), transformant le bloc de l'espace des pixels à l'espace des fréquences.

Le bit caché du message est récupéré à partir du coefficient DCT central [4, 4] du bloc. Ce coefficient contient le bit de poids faible du message caché.

Ce bit est extrait avec int(dct_block[4, 4]) & 1 (opération qui récupère uniquement le dernier bit).

Le bit extrait est ajouté à la chaîne binary_msg pour reconstruire le message binaire.

Identification du marqueur de fin :

À chaque ajout d'un bit dans binary_msg, le code vérifie si la chaîne se termine par le marqueur spécial '1111111111111110', qui indique la fin du message caché.

Si ce marqueur est trouvé, les 16 derniers bits sont supprimés (car ils sont le marqueur de fin) et la boucle s'arrête.

Conversion du message binaire en texte :

Une fois tous les bits du message récupérés, la fonction binary_to_text(binary_msg) est appelée pour convertir le message binaire en texte lisible.

💡 Cette méthode est efficace pour extraire des informations cachées dans une image en utilisant la DCT. Elle est discrète et robuste tant que l'image n'a pas subi de transformations destructrices (comme une compression excessive). La détection du marqueur de fin permet de récupérer précisément le message sans inclure de données inutiles.

In [ ]:
def extract_dct_watermark(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("Image non trouvée")
    h, w = img.shape
    binary_msg = ""
    for i in range(0, h, 8):
        for j in range(0, w, 8):
            block = img[i:i+8, j:j+8]
            if block.shape != (8, 8):
                continue
            dct_block = dct2(block)
            bit = int(dct_block[4, 4]) & 1
            binary_msg += str(bit)
            if binary_msg.endswith('1111111111111110'):
                binary_msg = binary_msg[:-16]
                break
    return binary_to_text(binary_msg)


Embedding de Watermark avec DWT (Discrete Wavelet Transform)
Dans cette cellule, on applique une méthode de tatouage invisible à une image en utilisant la Transformation en Ondelette Discrète (DWT), permettant d'incorporer un message secret dans l'image en modifiant les coefficients des ondelettes de l'image.

Voici les étapes importantes du code :

Chargement de l’image :

L'image est chargée en niveaux de gris avec cv2.imread(image_path, cv2.IMREAD_GRAYSCALE). Si l'image n'est pas trouvée, une exception (ValueError) est levée avec le message "Image non trouvée".

Transformation en ondelette (DWT) :

La DWT est appliquée à l'image avec pywt.dwt2(img, 'haar'). Cette transformation décompose l'image en plusieurs sous-bandes : LL (basse fréquence), LH, HL, et HH (hautes fréquences).

LL contient les informations les plus significatives de l'image (données à faible fréquence), tandis que LH, HL, et HH contiennent les détails à haute fréquence.

Conversion du message en binaire :

Le message à intégrer dans l'image est converti en binaire à l'aide de text_to_binary(message). Un marqueur de fin '1111111111111110' est ajouté à la fin du message pour marquer la fin du watermark.

Modification des coefficients DWT :

Le message binaire est intégré dans les coefficients de l'image, plus précisément dans les coefficients LL. Ces coefficients sont à la base de l'image et sont donc les plus adaptés pour cacher les informations sans trop perturber la qualité visuelle de l'image.

Les coefficients LL sont aplatis en un tableau unidimensionnel (LL_flat) afin de pouvoir insérer chaque bit du message binaire dans le bit de poids faible (LSB) de chaque coefficient.

Pour chaque bit du message, le bit de poids faible du coefficient LL est remplacé par le bit du message.

Vérification de la longueur du message :

Si le message est trop long pour l'image, une exception est levée avec le message "Message trop long pour l’image DWT."

Reconstruire l'image modifiée :

Une fois les coefficients modifiés, l'image est reconstruite à partir des coefficients modifiés avec pywt.idwt2(coeffs_modified, 'haar'). Cela applique l'inverse de la DWT (IDWT) pour récupérer l'image modifiée.

L'image est ensuite ajustée avec np.uint8(np.clip(img_reconstructed, 0, 255)) pour garantir que les valeurs des pixels restent dans la plage correcte [0, 255].

Enregistrement de l'image modifiée :

L'image avec le watermark est enregistrée à l'emplacement spécifié par output_path avec cv2.imwrite(output_path, img_reconstructed).

Un message de succès est affiché pour informer que le watermark a été appliqué avec succès.

💡 La DWT permet de cacher des informations dans les sous-bandes à faible fréquence de l'image, ce qui rend le watermark difficile à détecter visuellement. Cette méthode est robuste contre les transformations modérées (comme les rotations ou les changements d'échelle), mais elle peut être affectée par des compressions excessives ou des transformations trop fortes.










In [ ]:
def embed_dwt_watermark(image_path, message, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("Image non trouvée.")
    coeffs = pywt.dwt2(img, 'haar')
    LL, (LH, HL, HH) = coeffs
    msg_bin = text_to_binary(message) + '1111111111111110'
    data_index = 0
    LL_flat = LL.flatten()
    for i in range(len(LL_flat)):
        if data_index >= len(msg_bin):
            break
        LL_flat[i] = (int(LL_flat[i]) & ~1) | int(msg_bin[data_index])
        data_index += 1
    if data_index < len(msg_bin):
        raise ValueError("Message trop long pour l’image DWT.")
    LL_mod = LL_flat.reshape(LL.shape)
    coeffs_modified = (LL_mod, (LH, HL, HH))
    img_reconstructed = pywt.idwt2(coeffs_modified, 'haar')
    img_reconstructed = np.uint8(np.clip(img_reconstructed, 0, 255))
    cv2.imwrite(output_path, img_reconstructed)
    print("✅ Watermark DWT appliqué :", output_path)

Extraction de Watermark avec DWT (Discrete Wavelet Transform)
Dans cette cellule, nous extrayons un message caché dans une image en utilisant la Transformation en Ondelette Discrète (DWT), méthode utilisée pour insérer le watermark. Le processus d'extraction consiste à lire les coefficients DWT de l'image et à récupérer les bits du message caché.

Voici les étapes importantes du code :

Chargement de l’image :

L'image est chargée en niveaux de gris avec cv2.imread(image_path, cv2.IMREAD_GRAYSCALE). Si l'image ne peut pas être lue (par exemple si elle n'existe pas), une exception (ValueError) est levée avec le message "Image non trouvée".

Application de la DWT :

La DWT est appliquée à l'image avec pywt.dwt2(img, 'haar'). Cette transformation décompose l'image en plusieurs sous-bandes : LL, LH, HL, et HH.

Nous nous intéressons uniquement à la sous-bande LL, qui contient les informations à faible fréquence (les données les plus significatives de l'image).

Extraction des bits du message caché :

Les coefficients LL sont aplatis en un tableau unidimensionnel avec LL.flatten() afin de faciliter l'extraction des bits du message caché.

Le processus parcourt chaque coefficient val de LL_flat. Le bit de poids faible (LSB) de chaque coefficient est extrait avec int(val) & 1, ce qui permet d'obtenir un 0 ou un 1 à partir de chaque coefficient.

Les bits extraits sont ajoutés à la chaîne binary_msg.

Identification du marqueur de fin :

Après chaque ajout de bit, le code vérifie si la chaîne binary_msg se termine par le marqueur '1111111111111110', qui indique la fin du message caché.

Si ce marqueur est trouvé, les 16 derniers bits sont supprimés (car ce sont les bits du marqueur) et la boucle d'extraction s'arrête.

Conversion du message binaire en texte :

Une fois tous les bits récupérés, la fonction binary_to_text(binary_msg) est appelée pour convertir les bits en texte lisible, récupérant ainsi le message caché dans l'image.

💡 Cette méthode permet de récupérer un message caché dans l'image en utilisant les coefficients DWT. Le message est intégré dans la sous-bande LL, ce qui le rend difficile à détecter visuellement. Cependant, comme pour toute méthode de tatouage, une compression ou une transformation excessive de l'image peut affecter l'intégrité du watermark.

In [ ]:
def extract_dwt_watermark(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("Image non trouvée.")
    LL, _ = pywt.dwt2(img, 'haar')
    LL_flat = LL.flatten()
    binary_msg = ""
    for val in LL_flat:
        bit = int(val) & 1
        binary_msg += str(bit)
        if binary_msg.endswith('1111111111111110'):
            binary_msg = binary_msg[:-16]
            break
    return binary_to_text(binary_msg)


# ========== TEST GLOBAL ==========


In [ ]:
if _name_ == "_main_":
    message = "© Dhia & Med Amine 2025 - All rights reserved."

    # Watermark visible
    add_visible_watermark("ocean1.jpg", "output_visible.jpg", text=message)

    # Watermark invisible LSB
    encode_lsb("ocean1.jpg", message, "output_lsb.png")
    print("🔍 LSB decoded:", decode_lsb("output_lsb.png"))

    # Watermark DCT
    embed_dct_watermark("ocean1.jpg", message, "output_dct.png")
    print("🔍 DCT decoded:", extract_dct_watermark("output_dct.png"))

    # Watermark DWT
    embed_dwt_watermark("ocean1.jpg", message, "output_dwt.png")
    print("🔍 DWT decoded:", extract_dwt_watermark("output_dwt.png"))
